In [129]:
import cv2
import numpy as np
from detection2 import Yaw


# name = "IMG_3603"
# name = "IMG_3468"

# name = "IMG_4295"
# name = "IMG_4296"
# name = "IMG_4297"
# name = "IMG_4299"
# name = "IMG_4300"
# name = "IMG_4301"
name = "IMG_4302"

cap = cv2.VideoCapture('sh/mov/'+ name +'.MOV')
frames = []
angles = []
yaw_obj = Yaw()


c= 1
while True:


    ret, frame = cap.read()
    if not ret:
        break
    yaww = yaw_obj.predict_yaw(frame)
    angles.append(yaww)
    frames.append(frame)

    if (c% 100 ==0):
        print(c , yaww)
    c+=1

    
cap.release()
video_array = np.array(frames)

Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-2 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients


requirements: C:\Users\admin\.cache\torch\hub\requirements.txt not found, check failed.


Adding AutoShape... 


Model: "model_20"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_21 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_21[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                          

In [130]:
import os
import random
r = random.randint(0,12000)
# Path
# Create the directory
# 'GeeksForGeeks' in
# '/home / User / Documents'
os.mkdir("sh/mov/"+name+"__"+ str(r))

In [131]:
len(angles)

1543

In [132]:
def find_nearest(lst, num):
    if num is  None:
        return None

    nearest_num = lst[0]
    nearest_idx = 0

    if lst[0] is not None:
        min_diff = abs(num - lst[0])
    
    
    for i in range(1, len(lst) ):
        if lst[i] is  None:
            continue
        diff = abs(num - lst[i])
        if diff <= min_diff:
            nearest_num = lst[i]
            nearest_idx = i
            min_diff = diff
            
    return (nearest_num, nearest_idx)

In [133]:
def rotate_180(angle):
    return (angle + 180) % 360

In [134]:
def select_frame_from_period(frames ,angles , angle_range):
    print(angle_range ," -------> ", angles)
    a , b = angle_range
    angle_avg = (a+b) / 2
    angle , idx = find_nearest(angles , angle_avg)
    if angle is  None:
        return None
    if 280 > abs(angle - angle_avg) > 100:
        angle_avg = rotate_180(angle_avg)
        angle , idx = find_nearest(angles , angle_avg)
        angle = rotate_180(angle)
    return frames[idx] , angle





In [135]:
def frame_selection(angles , frames , num_frames = 30):
    selected_frames = []
    selected_angles = []
    angle_period = 360 // num_frames
    frame_period = frames.shape[0] // num_frames
    for i in range(num_frames):
        # print(i)
        half_period = angle_period // 2
        a = i * angle_period - half_period
        b = (i+1) * angle_period - half_period - 1
        c = i * frame_period
        d = (i+1) * frame_period -1
        
        frame_angle = select_frame_from_period( frames[c:d,:,:,:],angles[c:d] , angle_range = (a,b))

        if frame_angle is None:
            continue

        frame , angle = frame_angle
        
        
        selected_frames.append(frame)
        selected_angles.append(angle)

    return selected_frames , selected_angles




        

In [136]:
sel_frames , sel_angles = frame_selection(angles , video_array , 25)

(-7, 6)  ------->  [284.59123899390164, 284.61239721996844, 284.8566892682678, 285.1134378785199, 285.2502005127198, 283.70391172607606, 283.1280138750023, 283.6337127242333, 282.8333201352483, 281.1889941932983, 283.2809367425567, 281.7680578381843, 281.5353310118324, 282.30704704133336, 281.1214128623909, 280.98564402107064, 280.5582789318926, 283.0044437559015, 281.92748817079723, 281.7480061031587, 282.5092412036306, 283.2503255312986, 284.16949002977674, 282.85623372055954, 282.466624223066, 283.03867667626207, 281.0625314880497, 280.49088884635034, 281.35372646215023, 282.55407628892124, 280.4480652524895, 284.0541844425268, 282.6074542363709, 282.7620385480351, 283.5395619479543, 282.6133162483222, 282.0174366716868, 282.0025297784931, 283.9846701673931, 282.2340066797799, 282.129955540759, 283.6562574791974, 283.5412250996119, 283.680062404514, 282.98881798504055, 280.86543435624304, 279.1552721973426, 279.93265728588165, 280.0101065361208, 282.1585228171772, 281.7215920446462,

In [137]:
sel_angles

[98.54771128550237,
 67.96747832349149,
 68.01234158332261,
 66.39272428641675,
 65.11672966065316,
 66.03064643753828,
 56.47083726282108,
 118.2209572277103,
 120.15196186632042,
 121.29876130998399,
 135.6877485012375,
 133.94379140801357,
 120.9172903059617,
 112.93753795833219,
 229.667817548532,
 215.1227546092428,
 214.9056945349015,
 197.8494697765521,
 182.57803464394442,
 331.51601120094574,
 314.9416684136755,
 303.7847553208238,
 303.1230002536363,
 292.3601721413321,
 249.60362340343804]

In [138]:
sel_frames = np.array(sel_frames)
size = sel_frames.shape[2] , sel_frames.shape[1]

fps = 2


out = cv2.VideoWriter('sh/mov/'+ name+"__"+ str(r) +'/reco_.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

for f in sel_frames:
    out.write(f)
out.release() 


In [139]:

# List of images
images = sel_frames # Replace with your list of NumPy arrays

# Write images to disk
for i, image in enumerate(images):
    filename = f"sh/mov/"+ name+"__"+ str(r) +f"/image_{i}.png"
    cv2.imwrite(filename, image)

## real time detection

In [140]:
def find_main_angle(angle , expectation_idx = 0): # expectation_idx is previous angle_idx

    # old mode
    # important_angles = [0 , 30 , 90  , 150 , 180 , 200 , 270 , 340 , 360]
    # angle_names = ["front" , "front-left" , "left" , "back-left" , "back" , "back-right" , "right" , "front-right" , "front"]
    
    # correct mode
    # important_angles = [90 , 30 , 0  , 340 , 270 , 200 , 180 , 150 , 90]
    # angle_names = ["left" , "front-left" , "front" , "front-right" , "right" , "back-right" , "back" , "back-left" , "left"]


    # temp mode
    important_angles = [ 270 , 200 , 180 , 150 , 90 , 30 , 0  , 340 , 270]
    angle_names = [ "right" , "back-right" , "back" , "back-left" , "left" , "front-left" , "front" , "front-right" , "right"]

    nearest = find_nearest(important_angles , angle)
    if nearest is None:
        return "------------" , expectation_idx # nothing detected
    
    ang , idx = nearest
    if 280 > abs(ang - important_angles[expectation_idx]) > 100: # dont mistake on 180 degree
        angle2 = rotate_180(angle)
        ang , idx = find_nearest(important_angles , angle2)
    if idx != expectation_idx + 1  : # dont turn back
        idx = expectation_idx
    return angle_names[idx] , idx
    
    


In [141]:
font = cv2.FONT_HERSHEY_SIMPLEX
x, y = 50, video_array[0].shape[0] - 100
color = (150, 35, 200)
font_scale = 2
thickness = 8



In [142]:
import cv2
import os
import copy
# Set the directory containing the images
li = copy.deepcopy(video_array)

# Create a window to display the video
cv2.namedWindow('Video', cv2.WINDOW_NORMAL)

# Loop through each image and display it in the window
c= 0 
expectation_idx = 0
yawObj = Yaw()

detected_video = []

for img in li:
    if c % 20 == 0 :
        yaww = yawObj.predict_yaw(img)
        ang , idx = find_main_angle(yaww , expectation_idx = expectation_idx)
        expectation_idx = idx
    cv2.putText(img, "angle: "+str(ang), (x,y), font, font_scale, color, thickness)
    cv2.imshow('Video', img)
    detected_video.append(img)
    c+=1
    # Wait for 25 milliseconds between each frame (40 frames per second)
    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

# Release the window and close all windows
cv2.destroyAllWindows()

Using cache found in C:\Users\admin/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-5-2 Python-3.10.11 torch-2.0.0 CUDA:0 (NVIDIA GeForce RTX 3090, 24575MiB)

Fusing layers... 


requirements: C:\Users\admin\.cache\torch\hub\requirements.txt not found, check failed.


YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Model: "model_21"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_22 (InputLayer)          [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 224, 224, 64  1792        ['input_22[0][0]']               
                                )                                                                 
                                                                                                  
 block1_conv2 (Conv2D)          (None, 224, 224, 64  36928       ['block1_conv1[0][0]']           
                                )                                                          

In [143]:
detected_video = np.array(detected_video)
size = detected_video.shape[2] , detected_video.shape[1]

fps = 30


out = cv2.VideoWriter('sh/mov/RECO/___new3__'+ name+"_"+ str(r) +'reco.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, size)

for f in detected_video:
    out.write(f)
out.release() 